#Введение

Сегодня мы поговорим о задачах обработки естественного языка.
<br> Обработкой естественного языка (NLP) называется активно развивающаяся научная дисциплина, занимающаяся поиском смысла и обучением на основании текстовых данных.
<br> Решение задач nlp опять же дает вам целую кучу возможностей. Вы можете классифицировать текст: например, заставить алгоритм отвечать на определенные вопросы; производить анализ тональности текста (собственно, это то, чем вы занимались в домашнем задании): например, для рейтинговой системы фильмов или для предсказания цен на крипту.

# Теория


## Векторные представления слов (a.k.a. word embeddings)

Зачем это вообще надо. Текст сам по себе очень беспорядочный (то есть он может состоять из разного количества слов, причем слова все разной длины и тд). А алгоритмы типа нейронных сетей не предназначены для того, чтобы информация поступала в разных видах на ее входы. В связи с этим возникает необходимость создавать модели слов, которые можно использовать в алгоритмах машинного обучения. Такие модели представляют из себя цифры, а точнее последовательности цифр (a.k.a векторы)
<br>На сегодняшний день существует целая куча различных алгоритмов для перевода текста в более преемлимый вид. Среди них bag-of-words, tf-idf, word2vec, fasttext  и так далее. Важно отметить, что представление слов - это не просто присуждение номеров словам, а как бы отображение в цифрах сущности слова или целого предложения (то есть представление слова дает нам информацию о его лексическом и грамматическом значении). 
<br> Обсудим некоторые из вышеперечисленных алгоритмов


### Bag of Words (BoW)

Bage-of-words - это самый первый алгоритм, с которого все обычно начинают свой путь к познанию word embeddings. И мы тоже не будем нарушать традиции. 
<br> BoW - это один из способов извлечения признаков из текста (feature extraction). Смысл достаточно простой. Мы берем все знакомые нам слова и заводим счетчик **для каждого** из них. Берем предложение, смотрим на него, обновляем счетчики. В итоге получаем векторное представление для всего предложения. Обратите внимание, что в словарь можно включать не только слова, но и сочетания слов (или n-граммы) и считать уже количество сочетаний.

Допустим наш словарь состоит из слов: [егор, пошел, гулять, помидор, в, магазин]. И есть три предложения: 1. егор пошел гулять, 2. егор помидор, 3. егор пошел в магазин.
В этом случае представление предложений будет следующим:
1. [1, 1, 1, 0, 0, 0]
2. [1, 0, 0, 1, 0, 0]
3. [1, 1, 0, 0, 1, 1]

### tf-idf

Но в таком случае возникает проблема: если у нас текст большой и в нем есть много повторяющихся слов, которые не несут особого смысла (типа всякие союзы, частицы, слова-паразиты и тд).

Специально для этого придумали другой способ -- tf-idf (Term Frequency – Inverse Document Frequency). tf - это отношение числа вхождений какого-то слова к общему количеству слов в предложении, а idf - это логарифм отношения общего числа предложений к числу предложений, в которых встречается наше слово. Для того, чтобы получить репрезентацию слова нужно tf умножить на idf. Таким образом понижается важность частовстречающихся слов

In [34]:
import numpy as np
from math import log

text_base = ["егор пошел гулять",
             "егор помидор",
             "егор пошел в магазин"]

# составляем список слов
# делаем из него сет(множество), чтобы убрать дубли
# преобразуем опять в список, чтобы удобнее было работать
words = [i for ws in text_base for i in ws.split()]
vocab = set(words)
vocab = list(vocab)

# параметры векторов tf-idf
n_docs = len(text_base)
vec_len = len(vocab)

tf_vecs = np.empty([n_docs, vec_len]) # сюда будем складывать tf
vec = np.empty([0, vec_len]) # вспомогательный массив

# считаем tf
for i in range(n_docs):
    n_words = len(text_base[i].split()) # кол-во слов в предложении
    tf = np.asarray([]) # временное хранение
    for one in vocab:
        tf = np.append(tf, text_base[i].count(one)) # подсчитываем кол-во слов в предложении
    tf = tf / n_words # делим на кол-во слов каждый элемент в массиве
    tf_vecs[i] = np.vstack((vec, tf)) # заполняем массив tf
print("tf:")
print(tf_vecs)

idf = np.asarray([]) # сюда складываем idf

# считаем idf
for one in vocab:
    # считаем, в скольких предложениях есть наше слово
    cnt = 0
    for text in text_base:
        if text.count(one) > 0:
            cnt += 1
    # считаем
    idf = np.append(idf, log(n_docs/cnt))
print("idf:")
print(idf)

# считаем tf-idf
for i in range(n_docs):
    tf_vecs[i] = tf_vecs[i] * idf
    
tf_idf = tf_vecs
print("tf_idf:")
print(tf_idf)

tf:
[[0.         0.33333333 0.         0.33333333 0.33333333 0.        ]
 [0.5        0.         0.         0.         0.5        0.        ]
 [0.         0.25       0.25       0.         0.25       0.25      ]]
idf:
[1.09861229 0.40546511 1.09861229 1.09861229 0.         1.09861229]
tf_idf:
[[0.         0.13515504 0.         0.3662041  0.         0.        ]
 [0.54930614 0.         0.         0.         0.         0.        ]
 [0.         0.10136628 0.27465307 0.         0.         0.27465307]]


### Word2Vec

Теперь интереснее. BoW и tf-idf могут дать нам лишь относительное (относительно контекста) представление о слове. Но существуют методы, которые находят связи между многими контекстами и формируют абсолютное представление слова (то есть у нас представление слова всегда одинаково в любом предложении). Такими методами являются word2vec (от гугла) и fasttext (от фейсбука). Расскажу только про word2vec, так как fasttext не особо отличается (ну и про него легче искать информацию).

Простыми словами: word2vec — это инструмент (набор алгоритмов) для расчета векторных представлений слов, реализует две основные архитектуры — Continuous Bag of Words (CBOW) и Skip-gram. На вход подается корпус текста, а на выходе получается набор векторов слов.

Если не углубляться во всякие формулы, то идея проста: в одинаковом контексте слова должны иметь схожее значение (то есть они семантически близки друг к другу). Более формально задача стоит так: максимизация косинусной близости между векторами слов (скалярное произведение векторов), которые появляются рядом друг с другом, и минимизация косинусной близости между векторами слов, которые не появляются друг рядом с другом. Рядом друг с другом в данном случае значит в близких контекстах.

Пример:

Например, слова «анализ» и «исследование» часто встречаются в похожих контекстах, вроде «Ученые провели анализ алгоритмов» или «Ученые провели исследование алгоритмов». Word2vec анализирует эти контексты и делает вывод, что слова «анализ» и «исследование» являются близкими по смыслу. Так как подобные выводы word2vec делает на основании большого количества текста, выводы оказываются вполне адекватными. Скажем, когда я тренировал word2vec на коллекции из 10 000 небольших научных текстов (что, вообще-то, маловато), то для слова «Испания» на готовой модели получил следующий список наиболее близких по смыслу слов — Италия, Австралия, Нидерланды, Португалия, Франция. То есть, всё вполне четко, за исключением, разве что, Австралии.

После обучения модели можно проводить весьма интересные эксперименты:
* Искать семантически близкие слова:
<br>Enter word or sentence (EXIT to break): кофе
<br>— коффе 0.734483
<br>чая 0.690234
<br>чай 0.688656
<br>капучино 0.666638

* Складывать и вычитать разные слова:
<br>"жизнь" - "смерть" = "самообразование" (Word2Vec плохого не посоветует;))
<br>"король" - "мужчина" + "женщина" = "королева"
<br> и т.д.

* Оценка важности слов в запросе
<br>Принцип оценки прост. Надо определить, к какому кластеру тяготеет запрос в целом, а потом выбрать слова, максимально удалённые от центра этого кластера. Такие слова и будут главными, а остальные — уточняющими.
<br>1. Enter word or sentence (EXIT to break): **владимир путин**
<br>Importance владимир = 0.28982
<br>Importance путин = 1
<br>2. Enter word or sentence (EXIT to break): **никита путин**
<br>Importance никита = 0.793377
<br>Importance путин = 0.529835
* И многое другое...;)

На этом с векторными представлениями слов можно закончить, так как для начала самое главное просто интуитивно разобраться в этих вещах (если вас интересуют внутренности, то ссылки будут ниже).



## Recurrent Neural Network (RNN)

Наконец-то мы возвращаемся к нейронкам... Может вы знаете, а может и не совсем, но нейронки бывают не только с полносвязными слоями. Кроме полносвязной архитектуры существует множество других (сверточные, рекуррентные, генеративно-состязательные, со смешанными архитектурами и так далее)

Поговорим сначала о рекуррентных сетях. Давайте допустим, что у вас есть 2 галстука: один с цветочком, а другой с оленем. И вы каждое утро встаете в универ и думаете, какой сегодня надеть галстук. Спустя какое-то время вам надоело думать и вы просто решили написать нейросеть, которая бы принимала решение за вас. Основываясь на знаниях с прошлого занятия вы решили написать обычную полносвязную нейросеть с 3 входами (время года, день недели, ваше настроение по шкале от 1 до 10) и двумя выходами (наденете галстук с цветочком или с оленем). Получилось что-то вроде такого:
![alt text](https://churchman.nl/wp-content/uploads/2015/07/network-1-e1437307622159.png)

Далее, в течение некоторого времени вы собирали данные и наконец обучили вашу сеть. Но, как оказалось, она не работает (и это никак не связано с вашими умственными способностями). Просто вам стоило еще учитывать данные за день до этого (то есть, какой галстук вы надевали вчера). Чтобы решить эту задачу, вы принимаете решение доработать вашу нейронную сеть: вы добавляете в нее еще два входа. На первый новый вход вы подадите вчерашнее значение первого выхода вышей сети, а на второй новый вход подадите вчерашнее значение второго выхода сети. И таким образом, вы создадите рекуррентную нейронную сеть. 

Галстук мы вроде как смогли подобрать, но, что насчет чего-то более интересного. Обычно рекуррентные сети используют для предсказания временных рядов (например, какая будет цена на эфир завтра) или для генерации текста (например, чтобы ваш чатбот самостоятельно составлял предложения без вашей помощи) (https://www.theverge.com/2016/3/24/11297050/tay-microsoft-chatbot-racist). И это конечно супер-круто, но простая rnn не справится с такой задачей на отлично, так как она никак не сможет вспомнить, какой галстук вы надевали позавчера, и это будет иметь для вас серьезные последствия.

Рассмотрим пример более сложной задачи подробнее. Например, вы хотите, чтобы ваша нейросеть была четким пацаном и в любой непонятной ситуации могла подогнать вам четкую цитату, чтобы взбодрить корешей. Нейронка должна будет генерировать цитаты по очереди, одну за другой, чтобы вас не перестали уважать во дворе за повторы.

В качестве примера рассмотрим некоторые цитаты с сайта http://citaty.ru/pacanskie/:
1. "Не избегай драки, раны заживут быстрее, чем самооценка."

2. "Не беда, если нет друзей, беда если они фальшивые и продажные."

3. "Я не Минздрав — предупреждать не буду."

4. "Мне не важно прав он или нет, он мой брат, и я тебя за него разорву."

Так вот. Вы берете составляете словарь из всех слов, которые здесь присутствуют (**точка это тоже слово**). Число входов сети будет равно числу слов в словаре, умноженному на два, а число выходов равно просто числу слов в словаре. Обучим ее на этих примерах. Смысл обучения будет таков. Вам нужно зациклить эти 4 цитаты, чтобы за одной шла другая, за последней первая. Получается, что мы всегда знаем, какое слово будет следующим и какое было предыдущим. Зная это, можно обучить нейросеть. Но после обучения вы заметите, что нейросеть путает первую и вторую цитаты (так как первое слово это "не", а предыдущее за ним "." => входные данные одинаковы), поэтому она постоянно выдает вам либо первую, либо вторую. Поэтому здесь уже надо думать... Решение для такой задачи лежит в таком виде рекуррентных нейронных, как LSTM.



## Long Short-Term Memory (LSTM)

LSTM - это усовершенствованная rnn, которая может помнить дольше. Особо вдаваться в детали не буду (посмотрите видос внизу, там все круто рассказывается, если вам интересно), Скажу только, что в LSTM доабвляется обработка данных которые приходят с выходов на входы (то есть происходит учет более ранних событий). Именно благодаря этой обработке, rnn может запоминать дальше lstm. Ну и следовательно никаких проблем с пацанскими цитатами уже не будет.

Ну а теперь перейдем к практике...

# Практика

На практике мы закрепим то, что прошли на теории. То есть поработаем с word2vec и lstm.

Для начала установим gensim и скачаем готовую небольшую англоязычную модель word2vec, обученную гуглом на новостях

**Прежде чем начать зайдите по ссылке: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/ и добавьте файл к себе на диск(места не занимает)**



In [3]:
!pip install gensim

    100% |████████████████████████████████| 23.6MB 1.7MB/s 
    100% |████████████████████████████████| 1.4MB 14.6MB/s 
    100% |████████████████████████████████| 133kB 26.9MB/s 
    100% |████████████████████████████████| 61kB 22.5MB/s 
    100% |████████████████████████████████| 4.7MB 7.6MB/s 
    100% |████████████████████████████████| 552kB 23.5MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [1]:
from google.colab import drive # подключим возможность видеть наш гугл диск
drive.mount('/content/drive')
!ls drive/My\ Drive/ # проверка

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 asmlabs.zip
 Audream.rar
'Colab Notebooks'
 exlsv.rar
 formuls.txt
 GoogleNews-vectors-negative300.bin.gz
 IT_АРКТИКА_2016-1.pdf
 lab11.rar
 lab4.gdoc
 labs.tar.gz
 libnnet.tar.gz
 lll
 MTS.pdf
 mywiki.txt
 project1.rar
'PSEEWM160P146021 — копия.png'
 rl_trading
 site.rar
 tokenizedwiki.txt
 Untitled0.ipynb
'voprosi k ekzameny (1).docx'
'voprosi k ekzameny.docx.gdoc'
 Ассемблер.zip
'Копия Искусственные нейронные сети.ipynb'
 курсоваямоя.docx
'Новый точечный рисунок (2).bmp'
'План поездки.gsheet'
 рафтинг
 сети.docx


Ну теперь начнем. Подгрузим нашу модель в оперативную память и проверим работоспособность

In [0]:
import gensim
path = "drive/My Drive/GoogleNews-vectors-negative300.bin.gz" # путь к модели

model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True, unicode_errors="ignore")

Попробуем получить векторы нескольких слов

In [3]:
print(model['hello'])
print(model['leather'])
print(model['bag'])

[-0.05419922  0.01708984 -0.00527954  0.33203125 -0.25       -0.01397705
 -0.15039062 -0.265625    0.01647949  0.3828125  -0.03295898 -0.09716797
 -0.16308594 -0.04443359  0.00946045  0.18457031  0.03637695  0.16601562
  0.36328125 -0.25585938  0.375       0.171875    0.21386719 -0.19921875
  0.13085938 -0.07275391 -0.02819824  0.11621094  0.15332031  0.09082031
  0.06787109 -0.0300293  -0.16894531 -0.20800781 -0.03710938 -0.22753906
  0.26367188  0.012146    0.18359375  0.31054688 -0.10791016 -0.19140625
  0.21582031  0.13183594 -0.03515625  0.18554688 -0.30859375  0.04785156
 -0.10986328  0.14355469 -0.43554688 -0.0378418   0.10839844  0.140625
 -0.10595703  0.26171875 -0.17089844  0.39453125  0.12597656 -0.27734375
 -0.28125     0.14746094 -0.20996094  0.02355957  0.18457031  0.00445557
 -0.27929688 -0.03637695 -0.29296875  0.19628906  0.20703125  0.2890625
 -0.20507812  0.06787109 -0.43164062 -0.10986328 -0.2578125  -0.02331543
  0.11328125  0.23144531 -0.04418945  0.10839844 -0.28

Проверим размерность одного вектора

In [4]:
print(len(model['hello']))

300


Решим классическую задачу по анализу тональности отзывов imdb. Обратите внимание, что модель word2vec англоязычная и датасет тоже англоязычный.

Подгрузим датасет

Необходимо также знать о том, что в керасе по умолчанию в датасете imdb слова закодированы следующим образом: у каждого слова есть свой индекс (чем меньше индекс, тем чаще слово встречается в датасете). Переменная INDEX_FROM обозначает, с какого индекса мы начинаем отсчет (3, потому что в керасе по умолчанию индексы 0, 1 и 2 определены для спецсимволов, а именно 0 - <PAD\>, 1 - <START\>, 2 - <UNK\>).
    
А еще мы возьмем только часть датасета, потому что в гугл колабе не хватает оперативной памяти, чтобы обрабатывать весь

In [5]:
from keras.datasets import imdb

INDEX_FROM=3   # word index offset

train,test = imdb.load_data(index_from=INDEX_FROM) # грузим
train_x,train_y = train
test_x,test_y = test

test_x = train_x[train_x.shape[0] // 2:] # делим датасет
train_x = train_x[:train_x.shape[0] // 2]

test_y = train_y[train_y.shape[0] // 2:]
train_y = train_y[:train_y.shape[0] // 2]

train = [] # отчаянная попытка освободить память, чтобы ничего в конце не свалилось)
test = [] # если че, то это только для тех, кто запускает скрипт в ipython notebook (.ipnb)

Using TensorFlow backend.


 Ограничим длину каждого отзыва до 50 слов (иногда берется 80)

In [6]:
maxlen = 50

from keras.preprocessing.sequence import pad_sequences

train_x = pad_sequences(train_x, maxlen=maxlen)
test_x = pad_sequences(test_x, maxlen=maxlen)
print('train_x shape:', train_x.shape)
print('test_x shape:', test_x.shape)

train_x shape: (12500, 50)
test_x shape: (12500, 50)


Далее надо будет закодированные слова превратить в нормальные и перевести сразу в word2vec:

In [0]:
import numpy as np

def to_w2v(word_list): # эта функция для более удобного перевода предложения в word2vec
    global model
    if len(word_list) > 50: # почему-то при переводе в нормальные слова число слов в последовательностях поменялось, поэтому тут либо убираем лишние, либо добавляем символ <UNK>
        word_list = word_list[0:50]
    elif len(word_list) < 50:
        word_list.append("<UNK>")
    res = []
    for one in word_list:
        if one in model: # проверка на наличие данного слова в словаре
            res.append(np.asarray(model[one]))
        else:
            res.append(np.asarray([0.0 for _ in range(300)])) # если слова в словаре нет, то просто записываем нулевой вектор
            
    return np.asarray(res)

In [0]:
# https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset
word_to_id = imdb.get_word_index() # скачиваем индексы слов
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()} # составялем словарь
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()} # переворачиваем его, чтобы ключом стал индекс, а не слово

x_train = []
for one in train_x:
    s = ' '.join(id_to_word[id] for id in one) # составляем строку
    s = to_w2v(s.split()) # сразу переводим ее в word2vec-векторы
    x_train.append(s) # добавляем в список

x_test = []
for one in test_x:
    s = ' '.join(id_to_word[id] for id in one)
    s = to_w2v(s.split())
    x_test.append(s)
    
train_x = [] # очередная попытка освободить память
test_x = []

Приведем список к numpy-array

In [0]:
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

Отлично! Теперь создадим нашу первую lstm-сеть 

In [17]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

net = Sequential()

net.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5, input_shape=(maxlen, 300,)))# (max_length, the embedding_dimensions, )
net.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
net.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 219,777
Trainable params: 219,777
Non-trainable params: 0
_________________________________________________________________


In [18]:
x_test.shape

(12500, 50, 300)

In [19]:
test_y[43]

0

А теперь обучим нашу LSTM и сразу же проверим результаты обучения на тестовой выборке

In [20]:
net.fit(x_train, train_y,
          batch_size=50,
          epochs=30,
          validation_data=(x_test, test_y))
score, acc = net.evaluate(x_test, test_y,
                            batch_size=50)
print('Test score:', score)
print('Test accuracy:', acc)

Train on 12500 samples, validate on 12500 samples
Epoch 1/30
12500/12500 [==============================] - 45s 4ms/step - loss: 0.6208 - acc: 0.6521 - val_loss: 0.5269 - val_acc: 0.7378
Epoch 2/30
12500/12500 [==============================] - 42s 3ms/step - loss: 0.5777 - acc: 0.6955 - val_loss: 0.4915 - val_acc: 0.7676
Epoch 3/30
12500/12500 [==============================] - 42s 3ms/step - loss: 0.5524 - acc: 0.7182 - val_loss: 0.4735 - val_acc: 0.7680
Epoch 4/30
12500/12500 [==============================] - 42s 3ms/step - loss: 0.5417 - acc: 0.7266 - val_loss: 0.4939 - val_acc: 0.7652
Epoch 5/30
12500/12500 [==============================] - 42s 3ms/step - loss: 0.5144 - acc: 0.7482 - val_loss: 0.4493 - val_acc: 0.7869
Epoch 6/30
12500/12500 [==============================] - 42s 3ms/step - loss: 0.4865 - acc: 0.7639 - val_loss: 0.4399 - val_acc: 0.7929
Epoch 7/30
12500/12500 [==============================] - 42s 3ms/step - loss: 0.4747 - acc: 0.7687 - val_loss: 0.4338 - val_acc

Точность 82,5%.. Это не так уж и плохо. Вы можете поэкспериментировать с параметрами, подобавлять еще слоев (я просто очень сильно старался влезть в ограничение по оперативной памяти). Но самое главное, чему мы сегодня научились - это использование word2vec и LSTM. И все

На следующем занятии я бы хотел поговорить немного про картинки, сверточные сети, object detection и все такое)

# Домашнее задание

Вам необходимо улучшить то, что вы сделали в прошлом задании... Нужно заменить перевод слов в векторы на word2vec, используя gensim (https://nlpub.ru/Russian_Distributional_Thesaurus - здесь можно найти модель на 500мб и спокойно с ней работать). А также переработать вашу нейронку, а именно: перейти с dense-слоев на lstm (подсказка: конечный слой все равно лучше делать dense). 

После этого нужно подключиться к твиттеру через api (как - https://www.youtube.com/watch?v=o_OZdbCzHUA). 

Ну и в конце концов сделать так, чтобы вы вводили слово в вашего бота (например, "Путин"), а дальше бы по этому слову искались все твиты. Каждый твит бы обрабатывался вашей нейронкой. В итоге программа бы посчитала среднее по всем твитам и выдавала бы ответ (типа "Популярность "Путин" - N%")

На этом все) Подписывайтесь на канал, ставьте лайки, нажимайте колокольчик

# Полезные ссылки

https://machinelearningmastery.com/gentle-introduction-bag-words-model/ (подробнее о bag-of-words)

https://nlpub.ru/Russian_Distributional_Thesaurus (здесь можно почитать о word2vec и вообще о nlp, а еще можно скачать готовые модели word2vec)

http://nlpx.net/archives/179 (про word2vec)

https://youtu.be/EqWm8A-dRYg (тут предсказывают цену на биткоин с помощью анализа тональности постов на реддите)

https://youtu.be/WCUNPb-5EYI (хорошее объяснение rnn и lstm)

https://rare-technologies.com/word2vec-tutorial/ (туториал по генсиму и ворд2кеку)

https://habr.com/post/249215/ (куча примеров по word2vec, поможет понять принцип устройства)

Решение этой же задачи но без word2vec: 
 + https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py 
 + https://www.asozykin.ru/courses/nnpython-lab3